In [2]:
import pandas as pd
import numpy as np
import os
import json
from sklearn.model_selection import train_test_split

# 1. 데이터 로드
def load_data(file_path):
    columns = ["target", "id", "date", "flag", "user", "text"]
    data = pd.read_csv(file_path, encoding="ISO-8859-1", names=columns)
    return data[["target", "text"]]  # 필요한 열만 사용 (label, text)

# 2. Non-IID 분배
def split_non_iid(data, num_clients=5, dominant_classes=2):
    """
    Non-IID 방식으로 데이터를 나눕니다.
    각 클라이언트가 특정 클래스에 더 많은 비율의 데이터를 갖도록 분배.
    """
    clients = {f"client_{i}": [] for i in range(num_clients)}
    
    # 클래스별로 데이터 분리
    grouped = data.groupby("target")
    
    for class_label, group in grouped:
        # 클라이언트별로 데이터를 분배
        client_indices = np.random.choice(range(num_clients), size=num_clients, replace=False)
        splits = np.array_split(group.sample(frac=1, random_state=42), num_clients)
        
        for i, idx in enumerate(client_indices):
            if i < dominant_classes:  # 특정 클라이언트에 더 많은 데이터 분배
                clients[f"client_{idx}"].extend(splits[i].to_dict("records"))
            else:
                clients[f"client_{idx}"].extend(splits[i].to_dict("records"))

    return clients

# 3. 데이터 저장
def save_clients_to_json(clients, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for client_id, data in clients.items():
        file_path = os.path.join(output_dir, f"{client_id}.json")
        with open(file_path, "w") as f:
            json.dump(data, f, indent=2)

# 4. 실행 코드
if __name__ == "__main__":
    # 파일 경로 설정
    file_path = r"C:\Users\ahsld\Desktop\연합학습 트라이\training.1600000.processed.noemoticon.csv"  # Sentiment140 데이터 경로
    output_dir = "C:/Users/ahsld/Desktop/연합학습 트라이/clients_data"  # 분배된 데이터를 저장할 폴더
    
    # 데이터 로드
    print("Loading data...")
    data = load_data(file_path)
    
    # 데이터 분배
    print("Splitting data Non-IID...")
    clients = split_non_iid(data, num_clients=5, dominant_classes=2)
    
    # 데이터 저장
    print("Saving client data...")
    save_clients_to_json(clients, output_dir)
    
    print(f"Client data saved to {output_dir}/")


Loading data...
Splitting data Non-IID...
Saving client data...
Client data saved to C:/Users/ahsld/Desktop/연합학습 트라이/clients_data/
